In [1]:
import time
import csv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sage.rings.polynomial.toy_variety import coefficient_matrix
import itertools
from brial import *
from IPython.display import clear_output
import random
import copy
from collections import Counter
from tempfile import TemporaryFile

np.set_printoptions(suppress=True)

In [2]:
def moving_average(a, n) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def pascal_summation(n, a):
    value = 0
    for i in range(a+1):
        add = binomial(n, i)
        value = value + add
    return value

def return_uniqueterms_in_equations(equation_list):
    monomials_appearing = []
    for equation in equation_list:
        for term in equation.terms():
            if term not in monomials_appearing:
                monomials_appearing.append(term)
    monomials_appearing = badsort(monomials_appearing)
    return monomials_appearing

def badsort(mylist):
    for iterations in range(len(mylist)):            
        for i in range(len(mylist)-1):
            if mylist[i] > mylist[i+1]:
                mylist[i], mylist[i+1] = mylist[i+1], mylist[i]
    return mylist
    
def maximal_polynomial():
    term_list = []
    term = 1
    for i in range(len(B.gens())):
        term = term*B.gens()[i]
    term_list.append(term)
    return term_list

def second_best():
    second_best_list = []
    rji = list(B.gens())
    abc = maximal_polynomial()
    for r in rji:
        t = abc[0].set().vars()//r.lm()
        second_best_list.append(t)
    return second_best_list

In [3]:
def good_equation_generator(highest_degree, highest_terms, equations):
    v = BooleanPolynomialVector()
    l = [B.random_element(degree = highest_degree, terms = highest_terms) for _ in range(equations)]
    _ = [v.append(e) for e in l]
    output = []
    seen = set()
    for value in l:
    # If value has not been encountered yet,
    # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def term_limit_finder(degree, ring):
    var_count = len(ring.gens())
    term_limit = 0
    for i in range(degree+1):
        add = binomial(var_count, i)
        term_limit = term_limit + add
    return term_limit

def make_system_of_equations_consistent(equation_list): 
    MS = MatrixSpace(GF(2), len(equation_list), 1)
    matrix_answers = MS()
    x_values = []
    ans = []
    for i in range(NUMBER_OF_VARIABLES):
        value = GF(2).random_element()
        x_values.append(value)
    for f in equation_list:
        answer = f(*x_values)
        ans.append(answer)
    for i in range(len(equation_list)):
        matrix_answers[i,0] = ans[i]
        equation_list[i] = equation_list[i] + ans[i]
    return x_values, equation_list 

def generate_monomials(degree_max, ring):
    terms_limit = term_limit_finder(degree_max, ring)
    monomial_list = []
    v_1 = BooleanPolynomialVector()
    l = [ring.random_element(degree = degree_max, terms = terms_limit) for _ in range(1)]
    _ = [v_1.append(e) for e in l]
    f = l[0]
    for i in f:
        monomial_list.append(i)
    return monomial_list

def generate_monomials_high(degree, ring):
    monomials_to_degree = generate_monomials(degree, ring)
    unwanted = generate_monomials(degree-1, ring)
    '''Probably can make this step faster'''
    monomials_of_degree = [e for e in monomials_to_degree if e not in unwanted]
    return monomials_of_degree

def random_monomial_selector(monomial_list):
    output_monomial = monomial_list[ZZ.random_element(0, len(monomial_list))]
    return output_monomial

def detailed_equation_generator(): 
    ''' adjust the goddamn parameters yourself!'''
    num2degvarseqns = int(NUMBER_OF_VARIABLES/2)
    num2degvars = int(NUMBER_OF_VARIABLES/4)
    num1degvarseqns = 2*NUMBER_OF_VARIABLES
    num1degvars = int(NUMBER_OF_VARIABLES/3)
    generated_equation_list = [B.zero()]* EQUATIONS
    '''init'''
    monomial_list_2 = generate_monomials_high(2, B)
    monomial_list_1 = generate_monomials_high(1, B)
    for i in range(num2degvarseqns): # number of 2degree vars equations with 2 degree vars
        for j in range(num2degvars): # number of 2degree vars in each equation
            monomial_to_add = random_monomial_selector(monomial_list_2)
            generated_equation_list[i] = generated_equation_list[i] + monomial_to_add

    for i in range(num1degvarseqns):
        for j in range(num1degvars):
            monomial_to_add = random_monomial_selector(monomial_list_1)
            generated_equation_list[i] = generated_equation_list[i] + monomial_to_add

    # for i in generated_equation_list:
    #     print i
    #     time.sleep(0.1)
    #     clear_output(wait = True)

    monomial_list = generate_monomials(DEGREE_GAP, B)
    xvals, consistent_equation_list = make_system_of_equations_consistent(generated_equation_list)
    
    return consistent_equation_list, monomial_list

def simple_equation_generator(eqn_degree, terms, equations):
    generated_equation_list = good_equation_generator(eqn_degree, terms, equations)
    xvals, consistent_equation_list = make_system_of_equations_consistent(generated_equation_list)
    if len(consistent_equation_list) == EQUATIONS:
        return consistent_equation_list
    if len(consistent_equation_list) != EQUATIONS:
        generated_equation_list = good_equation_generator(eqn_degree, terms, equations)
        xvals, consistent_equation_list = make_system_of_equations_consistent(generated_equation_list)
        return consistent_equation_list
    
def find_simple_solutions(solutions):
    u = []; v = []
    for i in solutions:
        # finding 1 solutions
        if i[0].has_constant_part() and len(i[0]) == 2:
            a = i[0]-1
            if a not in u:
                u.append(a)
        if len(i[0]) == 1:
            b = i[0]
            if b not in v:
                v.append(b)
    d = {key: 1 for key in u}; e = {key: 0 for key in v}
    return d,e

In [32]:
def matrix_generator_density(equation_list_input,monomial_list_input, DENSITY):
    length_of_equations = len(equation_list_input)
    length_of_monomials = len(monomial_list_input)
    probability_mat = random_matrix(GF(2), length_of_equations, length_of_monomials, density = DENSITY)
    return probability_mat

def XL_random_prob(matrix, equation_list_input, monomial_list_input):
    new_equation_list = []
    columns = len(monomial_list_input)
    rows = len(equation_list_input)
    for i in range(rows):
        for j in range(columns):
            if matrix[i][j] == 1:
                monomial_to_mutiply = monomial_list_input[j]
                equation_to_mutiply = equation_list_input[i]
                new_equation = monomial_to_mutiply*equation_to_mutiply
                new_equation_list.append(new_equation)
    return new_equation_list

def select_stuff(equation_list, DENSITY):
    new_equation_list = []
    ran_mat = random_matrix(GF(2), 1, len(equation_list), density = DENSITY).numpy().tolist()[0]
    for i in range(len(ran_mat)):
        if ran_mat[i] == 1:
            new_equation_list.append(equation_list[i])
    return new_equation_list, ran_mat

def full_XL(equation_list, monomial_list):
    matrix =  matrix_generator_density(equation_list,monomial_list, 1)
    new_equation_list = XL_random_prob(matrix, equation_list, monomial_list)
    return new_equation_list

def partial_XL(equation_list, monomial_list, density):
    matrix =  matrix_generator_density(equation_list,monomial_list, density)
    new_equation_list = XL_random_prob(matrix, equation_list, monomial_list)
    return new_equation_list

def XL_reject(equation_list_input, monomial_list_input, reject):
    '''To make it work, increase equations or decrease terms'''
    matrix = matrix_generator_density(equation_list_input, monomial_list_input, 0.5)
    new_equation_list = []
    columns = len(monomial_list_input)
    rows = len(equation_list_input)
    for i in range(rows):
        for j in range(columns):
            if matrix[i][j] == 1:
                monomial_to_mutiply = monomial_list_input[j]
                equation_to_mutiply = equation_list_input[i]
                new_equation = monomial_to_mutiply*equation_to_mutiply
                if new_equation.degree() <= reject:
                    new_equation_list.append(new_equation)
    return new_equation_list

def XL_reject_full(equation_list_input, monomial_list_input, reject):
    '''To make it work, increase equations or decrease terms'''
    matrix = matrix_generator_density(equation_list_input, monomial_list_input, 1)
    new_equation_list = []
    columns = len(monomial_list_input)
    rows = len(equation_list_input)
    for i in range(rows):
        for j in range(columns):
            if matrix[i][j] == 1:
                monomial_to_mutiply = monomial_list_input[j]
                equation_to_mutiply = equation_list_input[i]
                new_equation = monomial_to_mutiply*equation_to_mutiply
                if new_equation.degree() <= reject:
                    new_equation_list.append(new_equation)
    return new_equation_list

def coefficient_matrix_generator(equation_list):
    try:
        A, v = Sequence(equation_list).coefficient_matrix()
        return A,v
    except Exception:
        print 'This failed'

def echelonize_the_matrix(the_matrix):
    echelonized_matrix = the_matrix.__copy__(); echelonized_matrix.echelonize(k = 10)
    return echelonized_matrix

def solver(equation_list):
    complex_matrix, terms = coefficient_matrix_generator(equation_list)
#     print 'Rank of matrix: ' + str(complex_matrix.rank())
    time_start = time.time()
    simple_matrix = echelonize_the_matrix(complex_matrix)
    time_end = time.time()
    simple_equations = simple_matrix*terms
    dict_one, dict_zero = find_simple_solutions(simple_equations)
#     print 'Solutions found: ' + str(len(dict_one) + len(dict_zero))
    return dict_one, dict_zero

In [26]:
'''Random fomulas'''

def standard_guassian_elimination():
    print time.time()
    del_cols = []
    del_rows = []
    for y in range(complex_matrix.ncols()):
        counts = list(complex_matrix.column(y)).count(B.one())
        if counts == 1:
            location = list(complex_matrix.column(y)).index(B.one())
            del_cols.append(y)
            del_rows.append(location)

    print del_rows, del_cols
    reduce_matrix = complex_matrix.__copy__()
    reduce_matrix = reduce_matrix.delete_columns(del_cols)
    reduce_matrix = reduce_matrix.delete_rows(del_rows)
    terms = terms.delete_rows(del_cols)
    print reduce_matrix.nrows(), reduce_matrix.ncols()
    simple_matrix = echelonize_the_matrix(reduce_matrix)
    # print simple_matrix*terms
    print time.time()
    
def XL_smart(equation_list, monomial_list, Density):
    matrix = matrix_generator_density(equation_list, monomial_list, Density)
    second_new_equation_list = []
    columns = len(monomial_list)
    rows = len(equation_list)
    for i in range(rows):
        for j in range(columns):
            if matrix[i][j] == 1:
                monomial_to_mutiply = monomial_list[j]
                equation_to_mutiply = equation_list[i]
                new_equation = monomial_to_mutiply*equation_to_mutiply
                second_new_equation_list.append(new_equation)
    return second_new_equation_list

def remove_least_common_monomials():
    removal = 0
    monomials_to_remove = []
    for monomial, counts in  reversed(monomial_counter(new_equation_list).most_common()):
        if removal <= difference - 3:
            removal = removal + counts
            monomials_to_remove.append(monomial)

    new_equation_list_2 = copy.copy(new_equation_list)

    # fastest
    for equation in new_equation_list:
        if len(set(equation.monomials()).intersection(set(monomials_to_remove))) != 0:
            new_equation_list_2.remove(equation)   

    monomial_counts = monomial_counter(new_equation_list_2);print len(monomial_counts), len(new_equation_list_2)
    dict_one, dict_zero  = %time solver(new_equation_list_2)
    
def monomial_counter(equation_list):
    y = []
    for equation in equation_list:
        for monomial in equation:
            y.append(monomial)
    monomial_counts = Counter(y)
    return monomial_counts

def testing_values():
    testing_value = 100
    monomial_counts = monomial_counter(new_equation_list)
    least_common_items = monomial_counts.most_common()[-testing_value:]
    dict_to_remove = dict((x,y) for x,y in least_common_items)
    items_to_remove = []
    for item in dict_to_remove:
        items_to_remove.append(item)

    print items_to_remove
    print sum(dict_to_remove.values())
    print monomial_counts
    
    proportion_counter = (float(number_equations - sum(dict_to_remove.values())))/(number_monomials - testing_value)
    print proportion_counter
    
def remove_useless_values(equation_list):
    monomial_counts = monomial_counter(equation_list)
    Y = monomial_counts.most_common()
    items_to_remove = []
    for x,y in Y:
        if y == 1:
            items_to_remove.append(x)
    second_new_equation_list = []
    for equation in equation_list:
        append_equation = True
        for monomial in equation.monomials():
            if monomial in items_to_remove:
                append_equation = False
                continue
        print equation, len(second_new_equation_list)
        clear_output(wait = True)
        if append_equation == True:
            second_new_equation_list.append(equation)
    return second_new_equation_list

In [27]:
def learning_two(equation_list, monomial_list):
    Q_table = random_matrix(GF(2), len(equation_list), len(monomial_list), density = 0.12)
    Q_table_new = Q_table.__copy__()
    new_equation_list = XL_random_prob(Q_table, equation_list, monomial_list)
    A,v = Sequence(new_equation_list).coefficient_matrix(sparse=False)
    benchmark = float(A.rank())/A.ncols()
    for row in range(Q_table_new.nrows()):
        for col in range(Q_table_new.ncols()):
            if Q_table_new[row][col] == 1:
                Q_table_new[row,col] = 0
                new_equation_list = XL_random_prob(Q_table_new, equation_list, monomial_list)
                A,v = Sequence(new_equation_list).coefficient_matrix()
                if float(A.rank())/A.ncols() < benchmark:
                    Q_table_new[row,col] = 1
                    print 'No changes made to row ' + str(row) + ', column ' + str(col) + '.'
                else:
                    benchmark = float(A.rank())/A.ncols() # if rank doesn't change from changing 1 to 0, leave it at 0
                    print 'Changed row ' + str(row) + ', column ' + str(col) + ' to 0.'
            if Q_table_new[row][col] == 0:
                Q_table_new[row,col] = 1
                new_equation_list = XL_random_prob(Q_table_new, equation_list, monomial_list)
                A,v = Sequence(new_equation_list).coefficient_matrix()       
                if float(A.rank())/A.ncols() <= benchmark:
                    Q_table_new[row,col] = 0 
                    print 'No changes made to row ' + str(row) + ', column ' + str(col) + '.'
                else:
                    benchmark = float(A.rank())/A.ncols()
                    print 'Changed row ' + str(row) + ', column ' + str(col) + ' to 1.'
            print "Considering equation " + str(equation_list[row]) + ", considering monomial " + str(monomial_list[col])
            print "Proportion = " + str(float(A.rank())/A.ncols())
            print "Density of Q_table = " + str(float(Q_table_new.density()))
    #         print Q_table_new
            clear_output(wait=True)
            
    return Q_table_new, A.rank(), float(Q_table_new.density())

In [28]:
NUMBER_OF_VARIABLES = 8
B = BooleanPolynomialRing(NUMBER_OF_VARIABLES,'x', order = 'degrevlex') # to be degraded soon
MONOMIAL_DEGREE = 1
EQUATION_DEGREE = 2
EQUATIONS = int(2*NUMBER_OF_VARIABLES)
TERMS = 4

consistent_equation_list = simple_equation_generator(EQUATION_DEGREE, TERMS, EQUATIONS)
monomial_list = generate_monomials(MONOMIAL_DEGREE, B) # generates monomials up to degree specified
monomial_list_full_1 = generate_monomials(EQUATION_DEGREE, B)
monomial_list_full_1.append(0)
monomial_list_full = monomial_list_full_1
print len(consistent_equation_list), len(monomial_list_full)

'''constants'''
# input tensor ixj
ENTRIES = 1500
POINTS = (TERMS + 1)*EQUATIONS
# layer 1 tensor, k
NODES_LAYER_1 = 1000
# layer 2 tensor, l
NODES_OUTPUT = len(monomial_list) + EQUATIONS

16 38


In [29]:
def pathway(first_density, second_density):
    selected_equation_list, ans_eqns = select_stuff(consistent_equation_list, first_density)
    selected_monomial_list, ans_mons = select_stuff(monomial_list, second_density)
    new_equation_list = full_XL(selected_equation_list, selected_monomial_list)
#     print len(new_equation_list), len(monomial_counter(new_equation_list))
    dict_one, dict_zero = solver(new_equation_list)
#     print dict_one, dict_zero
    if len(ans_eqns + ans_mons) == EQUATIONS + len(monomial_list):
        return len(dict_one) + len(dict_zero), ans_eqns + ans_mons
    if len(ans_eqns + ans_mons) < EQUATIONS + len(monomial_list):
        pass
    
def one_hot(equation_list):
    proxy = []
    for i in equation_list:
        if TERMS + 1 - len(i.monomials()) > 0:
            deficient = i.monomials()
            for j in range(TERMS + 1 - len(i.monomials())):
                deficient.append(0)
            proxy.append(deficient)
        if TERMS + 1 - len(i.monomials()) == 0:
            proxy.append(i.monomials())
    for equation in proxy:
        for element in equation:
            ind = monomial_list_full.index(element)  
            equation[equation.index(element)] = np.linspace(0.0, 1.0, num=len(monomial_list_full))[ind]
#         print equation
    return np.round(np.array(proxy),3)

def sigmoid(t):
    return 1/(1+np.exp(-t)) - 0.5

def sigmoid_squish(t):
    return 0.1*(1/(1+np.exp(-t))- 0.5)

def sigmoid_derivative(p):
    return p * (1 - p)

def select_from_numpy(numpy_matrix, list_to_select):
    selected_list = []
    for i in range(len(numpy_matrix)):
        if numpy_matrix[i] == 1:
            selected_list.append(list_to_select[i])
    return selected_list

In [30]:
equation_rep_list = []
answers_list = []
DATA_SET = 300

In [8]:
for j in range(DATA_SET):
    consistent_equation_list = simple_equation_generator(EQUATION_DEGREE, TERMS, EQUATIONS)
    print consistent_equation_list
    equation_representation = one_hot(consistent_equation_list).flatten()
    for i in range(100):
        try:
            solutions, answers = pathway(0.5, 0.5)
        except Exception:
            continue
        print solutions
        if solutions == NUMBER_OF_VARIABLES:
            print j, solutions
            clear_output(wait = True)
            answers_list.append(answers)
            equation_rep_list.append(equation_representation)
            break

[x4*x3 + x6*x0 + x7*x4 + x2 + 1, x5*x2 + x6*x1 + x6*x2 + x6*x5, x1*x0 + x3*x0 + x5*x4 + x6*x2, x3*x0 + x3*x2 + x4*x1 + x6*x4, x5*x3 + x6*x2 + x0 + x3, x1*x0 + x5*x4 + x7*x5 + x3, x4*x3 + x5*x4 + x0, x3*x2 + x6*x1 + x3 + x5, x3*x0 + x4*x0 + x5*x2, x2*x0 + x3*x0 + x6*x0 + x7 + 1, x3*x1 + x4*x3 + x5*x1 + x3, x3*x1 + x6*x0 + x7*x0 + x7*x1, x4*x0 + x4*x3 + x0 + x2, x3*x1 + x3*x2 + x7*x2 + x7*x6, x3*x2 + x6*x3 + x7*x4 + x4, x1*x0 + x2*x0 + x3 + x5]
[x5*x2 + x6*x2 + x7*x1 + x2, x5*x0 + x6*x2 + x7*x5 + x7 + 1, x5*x3 + x1 + x7 + 1, x2*x0 + x4*x3 + x5*x0 + x7*x6, x4*x2 + x5*x2 + x6*x4 + x7, x2*x1 + x4*x3 + x6*x2 + x0, x2*x0 + x6*x4 + x2 + x6, x2*x1 + x7*x2 + x4 + x6, x1*x0 + x5*x4 + x7*x0 + x7*x1, x1*x0 + x3*x2 + x5*x1 + x7*x6, x4*x2 + x5*x0 + x5*x2 + x6*x3 + 1, x4*x0 + x6*x4 + x4 + x7, x2*x1 + x4*x0 + x5*x2 + x6*x0 + 1, x4*x3 + x6*x1 + x7*x1 + x7*x2, x5*x3 + x7*x0 + x7*x4, x5*x1 + x6*x5 + x7*x2 + x2]
[x4*x1 + x4*x3 + x7*x2 + x4, x1*x0 + x3*x0 + x4*x0 + x5*x2, x2*x0 + x5*x4 + x6*x5 + x4 + 1, x3*

[x1*x0 + x7*x2 + x7*x3 + x2 + 1, x4*x0 + x6*x0 + x6*x2 + x1, x2*x0 + x4*x2 + x7*x4 + x4, x7*x4 + x7*x5 + x5 + x7 + 1, x1*x0 + x4*x3 + x6*x5 + x7*x6, x3*x1 + x6*x3 + x7*x3 + x7*x4, x2*x1 + x5*x0 + x7*x0 + x5, x4*x2 + x6*x1 + x6*x4 + x7*x5 + 1, x5*x3 + x6*x3 + x7*x2, x3*x1 + x7*x5 + x1 + x2 + 1, x3*x2 + x5*x4 + x7*x5 + x3, x5*x1 + x5*x2 + x6*x0 + x7*x6, x5*x0 + x5*x2 + x6*x1 + x7*x4, x5*x0 + x6*x0 + x0 + x4 + 1, x1*x0 + x3*x1 + x0 + x5 + 1, x6*x3 + x6*x4 + x6*x5 + x7*x6 + 1]
[x4*x0 + x4*x1 + x5*x3 + x6*x0 + 1, x6*x4 + x7*x0 + x7*x3 + x2, x6*x0 + x6*x5 + x7*x0 + x2, x5*x4 + x6*x2 + x7*x5 + x4 + 1, x5*x1 + x6*x0 + x7*x4 + x5 + 1, x6*x1 + x6*x2 + x7*x1 + x7*x2 + 1, x3*x2 + x7*x0 + x2, x4*x0 + x5*x4 + x7*x2 + x7*x3 + 1, x2*x1 + x4*x2 + x5*x2 + x5*x4 + 1, x3*x2 + x0 + x3, x3*x1 + x5*x1 + x7*x3 + x7*x4, x2*x1 + x1 + x3 + x5, x3*x0 + x3*x1 + x5*x1 + x1, x3*x1 + x5*x3 + x0 + x2, x4*x0 + x4*x1 + x5*x4 + x0 + 1, x1*x0 + x2*x0 + x4*x2 + x6]
[x4*x2 + x5*x0 + x6*x5 + x0 + 1, x1*x0 + x6*x2 + x7*x1 + x

KeyboardInterrupt: 

In [9]:
print len(answers_list), len(equation_rep_list)

0 0


In [10]:
'''Dont press this, will overwrite'''
outfile = TemporaryFile()
answers_array = np.asarray(answers_list, dtype=float, order=None)
equations_array = np.asarray(equation_rep_list, dtype=float, order=None)
print answers_array.shape
print equations_array.shape
np.save('answers_array_050319_3.npy', answers_array)
np.save('equations_array_050319_3.npy', equations_array)

(0,)
(0,)


In [11]:
X_array = np.load('equations_array_010319.npy')
Y_array = np.load('answers_array_010319.npy')
print X_array.shape, Y_array.shape

(1785, 100) (1785, 57)


In [38]:
answers_array = np.asarray(Y_array, dtype=float, order=None)
equations_array = np.asarray(X_array, dtype=float, order=None)

np.save('answers_array_050319_5.npy', answers_array)
np.save('equations_array_050319_5.npy', equations_array)



In [39]:
print X_array

[[1.    0.054 0.135 ... 0.892 0.946 0.973]
 [1.    0.189 0.324 ... 0.757 0.811 1.   ]
 [0.27  0.541 0.703 ... 0.838 0.865 1.   ]
 ...
 [0.486 0.568 0.649 ... 0.514 0.703 1.   ]
 [0.324 0.378 0.514 ... 0.514 0.784 1.   ]
 [0.027 0.189 0.27  ... 0.649 0.676 0.973]]


In [40]:
X_train, X_test = X_array[0:1500], X_array[1501:1784]
Y_train , Y_test = Y_array[0:1500].astype(float), Y_array[1501:1784].astype(float)

In [12]:
def slicing(slicer, array1, array2):
    return array1[slicer].reshape(1, POINTS), array2[slicer].reshape(1, NODES_OUTPUT)

def start():
    weights1 = np.random.rand(POINTS, NODES_LAYER_1)
    weights2 = np.random.rand(NODES_LAYER_1, NODES_OUTPUT)
    output = np.zeros(NODES_OUTPUT)
    return weights1, weights2, output

def feedforward(X, y, weights1, weights2):
    layer1 = sigmoid((np.dot(X, weights1)/POINTS))
    layer2 = sigmoid((np.dot(layer1, weights2)/NODES_LAYER_1))        
    return layer1, layer2

def backprop(X,y, weights1, weights2, layer1, output):
    d_weights1 = np.dot(X.T, np.dot(10*(y - output)*sigmoid_derivative(output), weights2.T)*sigmoid_derivative(layer1))/POINTS
    d_weights2 = np.dot(layer1.T, 10*(y - output)*sigmoid_derivative(output))/NODES_LAYER_1
    weights1 += d_weights1
    weights2 += d_weights2
    return weights1, weights2, np.mean(np.square(y - output))

def train_one():
    weights_one, weights_two, output = start()
    for epochs in range(10):
        for j in range(len(X_train)):
            X,y = slicing(j, X_train, Y_train)
            first_layer, NN_output = feedforward(X,y, weights_one, weights_two)
            weights_one, weights_two, loss = backprop(X,y, weights_one, weights_two, first_layer, NN_output)
            print NN_output, loss
            clear_output(wait = True)

# train_one()

def train_all():
    loss_list = []
    weights_one, weights_two, output = start()
    for epochs in range(1):
        X,y = X_train, Y_train
        first_layer, NN_output = feedforward(X,y, weights_one, weights_two)
        weights_one, weights_two, loss = backprop(X,y, weights_one, weights_two, first_layer, NN_output)
        print NN_output, loss
        loss_list.append(loss)
        
        if epochs % 5 == 0:
            layer1 = sigmoid((np.dot(X_test, weights_one)/POINTS))
            layer2 = sigmoid((np.dot(layer1, weights_two)/NODES_LAYER_1))  
            print np.mean(np.square(Y_test - layer2))
            
        clear_output(wait = True)
        
        
    return weights_one, weights_two

weights_one, weights_two = train_all()

np.save('weights_one_060319_1nodes.npy', weights_one)
np.save('weights_two_060319_1nodes.npy', weights_two)

NameError: global name 'X_train' is not defined

In [13]:
def small_matrix(dim_x, dim_y):
    A = np.round(sigmoid_squish(np.array(random_matrix(RDF, dim_x, dim_y))),5)
    A = np.asarray(A, dtype=float, order=None)
    return A

def weights_matrix(dim_x, dim_y):
    A = np.round(sigmoid(np.array(random_matrix(RDF, dim_x, dim_y))),5)
    A = np.asarray(A, dtype=float, order=None)
    return A

def solver_rank_to_mononials(equation_list, monomial_list):
    new_equation_list = full_XL(equation_list, monomial_list)
    complex_matrix, terms = coefficient_matrix_generator(new_equation_list)
    return float(complex_matrix.rank())/(complex_matrix.ncols()-1)

In [68]:
NUMBER_OF_VARIABLES = 15
B = BooleanPolynomialRing(NUMBER_OF_VARIABLES,'x', order = 'degrevlex') # to be degraded soon
MONOMIAL_DEGREE = 2
EQUATION_DEGREE = 2
EQUATIONS = int(1.5*NUMBER_OF_VARIABLES)
TERMS = 4

consistent_equation_list = simple_equation_generator(EQUATION_DEGREE, TERMS, EQUATIONS)
monomial_list = generate_monomials(MONOMIAL_DEGREE, B) # generates monomials up to degree specified

monomial_list_full_1 = generate_monomials(EQUATION_DEGREE, B)
monomial_list_full_1.append(0)
monomial_list_full = monomial_list_full_1

RESTRICTION_EQUATION = 20
RESTRICTION_MONOMIAL = 100
'''constants'''

# input tensor ixj
ENTRIES = 1500
POINTS = (TERMS + 1)*EQUATIONS

# layer 1 tensor, k
NODES_LAYER_1 = 20

# layer 2 tensor, l
NODES_OUTPUT = len(monomial_list) + EQUATIONS
print 'Nodes input: ' + str(POINTS)
print 'Nodes output: ' + str(NODES_OUTPUT)
print 'Equation length: ' + str(EQUATIONS)
print 'Monomial length: ' + str(len(monomial_list))

Nodes input: 110
Nodes output: 143
Equation length: 22
Monomial length: 121


In [69]:
weights_one = weights_matrix(POINTS, NODES_LAYER_1)
weights_two = weights_matrix(NODES_LAYER_1, NODES_OUTPUT)

def init(iterations):
    
    average_score_list = []
    all_weights_one = []
    all_weights_two = []
    
    for jj in range(iterations):
        
        weights_one = weights_matrix(POINTS, NODES_LAYER_1)
        weights_two = weights_matrix(NODES_LAYER_1, NODES_OUTPUT)

        score_list = []

        for rr in range(10):
            consistent_equation_list = simple_equation_generator(EQUATION_DEGREE, TERMS, EQUATIONS)
            one_hot_coded = one_hot(consistent_equation_list).flatten() # flatten and reshape are inverses

            layer1 = sigmoid(np.dot(one_hot_coded, weights_one))
            layer2 = sigmoid(np.dot(layer1, weights_two))

            layer2[layer2 < 0] = 0
            layer2[layer2 > 0] = 1

            equation_selection = layer2[ :EQUATIONS]
            monomial_selection = layer2[EQUATIONS: ]

            equation_list_XL = select_from_numpy(equation_selection, consistent_equation_list)
            monomial_list_XL = select_from_numpy(monomial_selection, monomial_list)

            score = solver_rank_to_mononials(equation_list_XL, monomial_list_XL)
            score_list.append(score)

        average_score = np.average(np.array(score_list))
        average_score_list.append(average_score)
        all_weights_one.append(weights_one)
        all_weights_two.append(weights_two)
        
        print average_score
        clear_output(wait = True)
        
    indexx = average_score_list.index(max(average_score_list))
    
    return all_weights_one[indexx], all_weights_two[indexx]
    
weights_one, weights_two = init(20)

0.36801139998241467


In [70]:
def generate_new_weights(weights_one, weights_two):
    
    list_weights_one = []
    list_weights_two = []

    for i in range(25):

        d_weights_one = small_matrix(POINTS, NODES_LAYER_1)
        d_weights_two = small_matrix(NODES_LAYER_1, NODES_OUTPUT)

        new_weights_one = weights_one + d_weights_one
        new_weights_two = weights_two + d_weights_two

        list_weights_one.append(new_weights_one)
        list_weights_two.append(new_weights_two)
    
    return list_weights_one, list_weights_two

In [71]:
list_weights_one, list_weights_two = generate_new_weights(weights_one, weights_two)

In [72]:
def test_new_weights(list_weights_one, list_weights_two):
    average_score_list = []

    for i in range(len(list_weights_one)):

        weights_to_test_one = list_weights_one[i]
        weights_to_test_two = list_weights_two[i]

        score_list = []

        for rr in range(20):
            consistent_equation_list = simple_equation_generator(EQUATION_DEGREE, TERMS, EQUATIONS)
            one_hot_coded = one_hot(consistent_equation_list).flatten() # flatten and reshape are inverses

            layer1 = sigmoid(np.dot(one_hot_coded, weights_to_test_one))
            layer2 = sigmoid(np.dot(layer1, weights_to_test_two))

            layer2[layer2 < 0] = 0
            layer2[layer2 > 0] = 1

            equation_selection = layer2[ :EQUATIONS]
            monomial_selection = layer2[EQUATIONS: ]
            
            equation_list_XL = select_from_numpy(equation_selection, consistent_equation_list)
            monomial_list_XL = select_from_numpy(monomial_selection, monomial_list)

            if len(equation_list_XL) > RESTRICTION_EQUATION or len(monomial_list_XL) > RESTRICTION_MONOMIAL:
                continue
                
            else:
                score = solver_rank_to_mononials(equation_list_XL, monomial_list_XL)
                score_list.append(score)

        average_score = np.average(np.array(score_list))
        average_score_list.append(average_score)

        print average_score
        clear_output(wait = True)

    indexx = average_score_list.index(max(average_score_list))
    
    return list_weights_one[indexx], list_weights_two[indexx]



In [73]:
weights_one, weights_two = test_new_weights(list_weights_one, list_weights_two)

0.5510860528627781


In [ ]:
'''Dont press this accidently, will overwrite'''
for i in range(100):
    weights_one, weights_two = test_new_weights(list_weights_one, list_weights_two)
    list_weights_one, list_weights_two = generate_new_weights(weights_one, weights_two)

0.8389600395450849


In [ ]:
np.save('Stockfish_07032019_w1n15s1941m2662.npy', weights_one)
np.save('Stockfish_07032019_w2n15s1941m2662.npy', weights_two)

In [59]:
print weights_one

[[ 0.08552  0.19152  0.12524 ... -0.23559 -0.01939 -0.06391]
 [-0.07183 -0.06729 -0.26412 ... -0.17091  0.07703  0.04512]
 [-0.12505 -0.16684 -0.04639 ...  0.02674  0.21213  0.20578]
 ...
 [-0.1761  -0.16531 -0.20663 ... -0.15615  0.19132 -0.02525]
 [-0.17385  0.10362  0.07611 ...  0.12791 -0.36788 -0.07148]
 [-0.09543 -0.21488 -0.22041 ...  0.025   -0.06201 -0.06168]]


In [60]:
NN_result_list = []
random_result_list = []

for test in range(50):
    consistent_equation_list = simple_equation_generator(EQUATION_DEGREE, TERMS, EQUATIONS)
    one_hot_coded = one_hot(consistent_equation_list).flatten() # flatten and reshape are inverses

    layer1 = sigmoid(np.dot(one_hot_coded, weights_one))
    layer2 = sigmoid(np.dot(layer1, weights_two))

    layer2[layer2 < 0] = 0
    layer2[layer2 > 0] = 1

    equation_selection = layer2[ :EQUATIONS]
    monomial_selection = layer2[EQUATIONS: ]

    equation_list_XL = select_from_numpy(equation_selection, consistent_equation_list)
    monomial_list_XL = select_from_numpy(monomial_selection, monomial_list)

    new_equation_list = full_XL(equation_list_XL, monomial_list_XL)
    dict_one, dict_zero = solver(new_equation_list)

    NN_result = len(dict_one) + len(dict_zero)
    NN_result_list.append(NN_result)

    abc = []
    for jj in range(15):
        equation_list_rand = select_stuff(consistent_equation_list, float(len(equation_list_XL))/len(consistent_equation_list))[0]
        monomial_list_rand = select_stuff(monomial_list, float(len(monomial_list_XL))/len(monomial_list))[0]

        new_equation_list = full_XL(equation_list_rand, monomial_list_rand)
        dict_one, dict_zero = solver(new_equation_list)

        random_result =  len(dict_one) + len(dict_zero)
        abc.append(random_result)

    random_result_averaged = np.average(np.array(abc))
    random_result_list.append(random_result_averaged)
    print NN_result, random_result_averaged
    clear_output(wait = True)

68 116.0


In [61]:
print len(NN_result_list)
print len(random_result_list)

print np.average(np.array(NN_result_list))
print np.average(np.array(random_result_list))

50
50
128.22
104.48533333333334


In [62]:
print NN_result_list
print random_result_list

[159, 1, 0, 159, 168, 0, 168, 162, 1, 159, 149, 149, 172, 146, 167, 17, 159, 159, 2, 158, 0, 170, 165, 137, 154, 170, 155, 164, 168, 160, 161, 155, 174, 0, 167, 169, 161, 161, 170, 162, 159, 148, 158, 3, 154, 79, 152, 157, 155, 68]
[65.0, 78.66666666666667, 34.266666666666666, 116.0, 104.73333333333333, 10.533333333333333, 126.13333333333334, 77.53333333333333, 70.46666666666667, 50.2, 61.4, 124.46666666666667, 121.93333333333334, 127.06666666666666, 156.13333333333333, 105.2, 79.8, 156.0, 51.46666666666667, 92.8, 94.2, 147.73333333333332, 147.13333333333333, 122.33333333333333, 95.93333333333334, 149.0, 114.66666666666667, 106.46666666666667, 155.66666666666666, 95.53333333333333, 156.13333333333333, 86.26666666666667, 107.86666666666666, 47.2, 132.86666666666667, 121.66666666666667, 119.6, 111.73333333333333, 121.26666666666667, 125.8, 152.53333333333333, 95.53333333333333, 135.66666666666666, 24.0, 137.0, 73.73333333333333, 88.66666666666667, 154.33333333333334, 77.93333333333334, 1